# 离子阱单/双量子比特门

*版权所有 (c) 2021 百度量子计算研究所，保留所有权利。*


## 内容概要
本教程中，我们将介绍如何使用量脉在离子阱量子计算中进行单量子比特门和 Mølmer-Sørensen 门操作。本教程的大纲如下：


- 背景介绍
- 准备工作
- 绕 X/Y 轴旋转的单量子比特门操作
- 离子阱 Mølmer-Sørensen 门背景介绍
- 离子阱 Mølmer-Sørensen 门脉冲生成
- 总结
- 参考文献

## 背景介绍

离子阱是最早被提出用于进行量子计算的平台之一。近些年来，离子阱量子计算技术得到了长足的发展，单双量子比特门保真度已经达到 99.9999% 和 99.9% \[1\]。和其他诸如超导量子计算硬件平台相比，离子阱量子计算有几大优势：其一是量子比特单元是全同粒子，量子比特频率固定；其二是单/双量子比特保真度很高且量子比特相干时间长；其三是离子阱中的量子比特通过库伦相互作用连接，可以做到量子比特之间的全连通 \[2\]。

离子阱量子计算是将束缚在势阱中的离子作为量子比特单元，通过外加驱动的形式实现对每个量子比特的操作。不同于超导量子比特采用的微波脉冲形式，离子阱中常用的外加驱动是激光脉冲 \[3\]，通过下图（a）中的 Raman 过程实现图（b）离子内部能级的跃迁，从而构成所需的量子操作。鉴于此，我们研发了量脉离子阱，用户可以通过量脉离子阱生成离子阱量子控制中所需的激光脉冲。

**Raman 过程和等效哈密顿量**

我们选定离子内部两个超精细能级作为量子比特的 $|0\rangle$ 和 $|1\rangle$ 态， 因为实验上选定的两个超精细能级之间存在电偶极禁闭，所以要将两个态耦合起来需要通过高能级上的中间态 $|e\rangle$ 过渡，实验上通常会采取两束激光作用在一个离子上形成双光子 Raman 构型实现 $|0\rangle$ 态和 $|1\rangle$ 态之间的等效耦合。

![Raman](figures/Raman.png)

双光子 Raman 过程通过旋波近似（Rotating Wave Approximation, RWA）和绝热消除（Adiabatic Elimination）去掉中间过渡态 $|e\rangle$ 的影响后得到等效哈密顿量：

$$
\hat{H}_{eff}=\frac{ \Omega(t)}{2}\left(\hat{\sigma}_{+} e^{i\Delta\phi}+\hat{\sigma}_{-} e^{-i\Delta\phi}\right),
$$

其中 $\hat{\sigma}_+=(\hat{\sigma}_x+i\hat{\sigma}_y)/2，\hat{\sigma}_-=(\hat{\sigma}_x-i\hat{\sigma}_y)/2$，$\hat{\sigma}_x,\hat{\sigma}_y$ 是 Pauli 矩阵。$\Delta\phi=\phi_1-\phi_0$ 是两束作用在离子上 Raman 激光的相对相位，Rabi 频率 $\Omega(t)$ 正比于两束激光的振幅的乘积 $\Omega(t)\propto E_1(t)\cdot E_0(t)$，一般实验采用的 Raman 激光具有 $|E_1|=|E_0|$ 的特性。在离子阱量子调控中，激光参数为 $(E,\omega,\phi)$，受限于激光器的特性，我们一般初始时刻会固定激光的频率 $\omega$ 和相对相位 $\Delta\phi $，通过调节激光振幅 $E$ 实现对离子阱量子比特的操控。因为 $\Omega(t)$ 和 $E(t)$ 之间具有简单的线性关系，后续离子阱量子调控指的是调控 $\Omega(t)$。

**离子阱单量子比特旋转门**

离子阱量子控制中能实现的单量子比特旋转门矩阵形式为

$$
R(\theta,\Delta\phi) = \begin{bmatrix} 
\cos \frac{\theta} { 2 } & -\mathrm{i} \mathrm{e}^{\mathrm{i} \Delta\phi} \sin \frac{\theta} { 2 } \\
-\mathrm{ie}^{-\mathrm{i} \Delta\phi} \sin \frac{\theta} { 2 } & \cos \frac{\theta} { 2 }   
\end{bmatrix}.
$$

当选取两束激光的相对相位为 $\Delta\phi=0$ 时，对应绕 X 轴的旋转

$$
R_x(\theta)=\begin{bmatrix} 
\cos \frac{\theta} { 2 } &  -i \sin \frac{\theta} { 2 } \\
-i \sin \frac{\theta} { 2 } & \cos \frac{\theta} { 2 } 
\end{bmatrix}.
$$

当选取两束激光的相对相位为 $\Delta\phi=-\pi/2$ 时，对应绕 Y 轴的旋转

$$
R_y(\theta)=\begin{bmatrix} 
\cos \frac{\theta} { 2 } &  - \sin \frac{\theta} { 2 } \\
 \sin \frac{\theta} { 2 } & \cos \frac{\theta} { 2 } 
\end{bmatrix}.
$$

下面我们将先后介绍如何在量脉中实现离子阱单双量子比特门操作。

## 准备工作

首先，在成功安装量脉后，用户需要从量脉（Quanlse）和其它常用的 Python 库导入以下模块：

In [ ]:
# Import necessary packages
from math import pi
from Quanlse.remoteOptimizer import remoteIonOptimize1Qubit as runIonOptimize1Qubit
from Quanlse.Utils import Plot

在使用量脉云服务之前，用户需要从 http://quantum-hub.baidu.com 获取一个 token，并使用以下命令将任务提交到量脉的服务器上：

In [ ]:
from Quanlse import Define

Define.hubToken = ""

## 绕 X/Y 轴旋转的单量子比特门操作
**绕 X 轴旋转门操作**

我们以下面的代码为例。首先定义离子阱单量子比特旋转门中必要的函数参数，包括旋转轴 ``axis``、旋转角度 ``theta``、量子门时间 ``tgate``：

In [ ]:
# Choose the rotating axis, where we have ionRx and ionRy
axis = "ionRx"

# The rotating angle
theta = pi / 3

# The gate time with units microsecond
tgate = 2

通过调用函数 `runIonOptimize1Qubit()`，传入定义好的参数:

In [ ]:
amp, infid, uReal = runIonOptimize1Qubit(axis, theta, tgate)
print(infid)
print(uReal)

我们能计算出生成绕 X 轴旋转 $\theta$ 角度的量子门所需的哈密顿量脉冲信息 `ham` 和脉冲作用后的量子门失真度 `infid` 并 打印脉冲生成的真实酉矩阵。通过 `Plot.plotPulse()` 函数：

In [ ]:
Plot.plotPulse([[tgate/2, tgate]], [[amp, amp]],
               title=[r'Square pulse for $\phi=0$'],
               xLabel=r'Time ($\mu$s)', yLabel=['Rabi frequency (a.u)'], color=['blue'])

我们可以将生成的激光脉冲可视化。

**绕 Y 轴旋转门操作**

同样地，我们可以选择绕 Y 轴的旋转门，通过定义如下参数：

In [ ]:
# Choose the Y rotating axis
axis = "ionRy"

# The rotating angle
theta = pi / 2

# The gate time with units microsecond
tgate = 2

调用 `runIonOptimize1Qubit()`，可以求得绕 Y 轴旋转的量子门失真度和脉冲参数：

In [ ]:
amp, infid, uReal = runIonOptimize1Qubit(axis, theta, tgate)
print(infid)
print(amp)
print(uReal)

我们能计算出生成绕 Y 轴旋转 $\theta$ 角度的量子门所需的哈密顿量脉冲信息 `ham` 和脉冲作用后的量子门失真度 `infid` 并打印脉冲生成的真实酉矩阵。通过 `Plot.plotPulse()` 函数，我们可以将生成的激光脉冲可视化。

In [ ]:
Plot.plotPulse([[tgate / 2, tgate]], [[amp, amp]],
               title=[r'Square pulse for $\phi=-\pi/2$'],
               xLabel=r'Time ($\mu$s)', yLabel=['Rabi frequency (a.u)'], color=['blue'])

可以看到，当我们选取不同的旋转轴时，等效哈密顿量中激光作用的通道将有所差别，选取 $X$ 轴，激光脉冲作用在 $\hat\sigma_x$ 上，选取 $Y$ 轴，激光脉冲作用在 $\hat\sigma_y$ 上。

## 离子阱 Mølmer-Sørensen 门背景介绍

要实现通用量子计算，除了实现单量子比特旋转门外，还需要实现双量子比特的纠缠门。下面我们介绍离子阱量子计算中的原生双量子比特门 Mølmer-Sørensen 门的原理 \[4\] 以及如何用量脉生成相应的脉冲。
 
**离子阱 Mølmer-Sørensen 门原理**

![Raman](figures/MSgate.png)

离子阱量子计算中，两个量子比特通过离子链的集体振动—声子模来传递信息。在四束激光的作用下，体系的等效哈密顿量可以写成

$$
\hat{H}_{M S}= \hbar \sum_{j=\{m,n\}} \Omega_{j}(t) \hat{{\sigma}}_{x}^{j}\sum_{k=1}^{N}\eta_{j,k}e^{i\mu t}\left(\hat{a}_k e^{-i \omega_{k} t}+\hat{a}_k^{\dagger} e^{i \omega_{k} t}\right) + \rm H.c.,
$$

其中 $j,k$ 分别代表离子和声子的指标，$\{m,n\}$ 是激光作用的离子位置标号。$N$是阱中离子总数，也是参与传递信息的声子模个数。$\mu$ 是激光的矢谐，失谐选取和门时间 $t_g$ 满足关系 $\mu=2\pi/t_g$。$\hat{\sigma}_x^j$ 是第 $j$ 个离子的 Pauli 矩阵，$\hat a_k^\dagger,\ \hat a_k$ 分别是第 $k$ 个声子的产生湮灭算符。$\Omega_j$ 是我们需要优化的脉冲，一般情况下 $\Omega_m(t)=\Omega_n(t)$。$\eta_{j,k}$ 是振动声子模的 Lamb-Dicke 系数，$\omega_k$ 代表声子的振动频率。为了后续方便，我们设 $\hbar=1$。

**从哈密顿量 $\hat{H}_{MS}$ 到 $U$ 算符**

构成哈密顿量的关键参数 $\{\eta_{j,k},\omega_k\}$ 和一维离子链束缚势 $V(x,y,z)=\frac{1}{2}\omega_z^2 z^2-\frac{1}{2}\omega_{xy}^2(x^2+y^2)$ 以及离子阱中离子数目 $N$ 有关，一维离子链中，我们需要保证 $\omega_z \ll \omega_{xy}$。且离子阱量子计算中 Lamb-Dicke 系数 $\eta_{j,k}\ll 1$，使得上述等效哈密顿量含时演化可以展开到 Magnus 二阶：

$$
U_{g}(t_g)=\exp \left[-i\sum_{j=\{m,n\}}\hat{\sigma}_{x}^{j}\sum_{k=1}^{N}\left(\alpha_{j, k}(t_g) \hat{a}_{k}^{\dagger}-\alpha_{j, k}^{*}(t_g) \hat{a}_{k}\right) +i\chi_{m,n}(t_g)\hat{\sigma}_{x}^{m}\hat{\sigma}_{x}^{n}+O(\eta^3)\right],
$$

其中 
$$
\alpha_{j, k}(t_g)=\int_0^{t_g} dt f(\eta_{k,j},\Omega_j(t),\omega_k),
$$

$$
\chi_{m,n}(t_g)=\int_0^{t_g} dt\int_0^{t} dt_1 g(\eta_{k,j},\Omega_m(t), \Omega_n(t_1), \omega_k).
$$

积分函数 $\{f,g\}$ 和选择的激光脉冲息息相关。脉冲 $\Omega_j(t)$ 可以设置成切片脉冲 $[A_1,A_2,\cdots,A_{n_{2N+1}}]$ ($A_n$是第 $n$ 切片的脉冲振幅）或者傅里叶正弦基 $\Omega_j(t)=\sum_{n=1}^{2N+1}A_n \sin(2\pi n t/t_g)$ 的形式。通过脉冲 $\Omega_j(t)$ 中 $A_n$ 的调整，可以得到一组脉冲 $\Omega_j(t)$，使得 $\alpha_{j, k}(t_g)=0,\chi_{m,n}(t_g)=\frac{\pi}{4}$，从而形成 Mølmer-Sørensen 门：

$$
MS=U(t_g)=e^{i\frac{\pi}{4}\hat{\sigma}_x^m\otimes\hat{\sigma}_x^n}=\frac{1}{\sqrt{2}}\left(\begin{array}{cccc}
1 & 0 & 0 & i \\
0 & 1 & i & 0 \\
0 & i & 1 & 0 \\
i & 0 & 0 & 1
\end{array}\right).
$$

Mølmer-Sørensen 门和量子计算中常用的 CNOT 门之间只相差局域的酉变换，等效电路如下：

![MSgate](figures/MS-CNOT.PNG)

## 离子阱 Mølmer-Sørensen 门脉冲生成

想要生成离子阱 Mølmer-Sørensen 门脉冲，我们首先需要从量脉中导入求解离子阱 Mølmer-Sørensen 门相应的模块：

In [ ]:
# Import Molmer-Sorensen gate ion optimizer module
from Quanlse.remoteOptimizer import remoteIonMS as runIonMS
from Quanlse.Utils import Plot

第一步，定义离子阱中量子比特的信息，包括离子阱中离子数目、阱中离子的种类（质量）、离子阱的横纵束缚势、以及用户想选择用作媒介的振动声子模式：

In [ ]:
# Set ion number in the trapped potential 
ionNumber = 4

# Set atom mass or atom species in the trapped potential
atomMass = 40

# Set trapped potential parameter
omegaZ = 2 * pi * 0.2e6
omegaXY = 2 * pi * 4.1e6

# Choose transverse or axial phonon mode
phononMode = "transverse"

同时，要对两个离子进行 Mølmer-Sørensen 门操作，我们指明需要操作的两个离子指标以及作用的门时间，以及用户想采用的激光脉冲波形（用户可选择方波 `squareWave` 和正弦波 `sinWave`）：

In [ ]:
# Set operating ion indexs
ionM = 1
ionN = 2

# Set the gate time
tgate=50

# Choose the pulse waveform, where we support squareWave and sinWave
pulseWave = "squareWave"

调用 `runIonMS()` 函数，并传入上面的离子阱参数：

In [ ]:
res, unitary = runIonMS(ionNumber=ionNumber,
               atomMass=atomMass,
               tg=tgate,
               omega=(omegaXY, omegaZ),
               ionIndex=(ionM, ionN),
               pulseWave=pulseWave,
               phononMode=phononMode)

其中 `res` 包含了离子阱量子系统中量子比特位置信息、集体振动声子模频率、系统 Lamb-Dicke 系数矩阵、形成 Mølmer-Sørensen 门脉冲序列以及脉冲对应的保真度和真实量子门矩阵形式。用户通过如下命令可以方便地打印出这些相关参数：

In [ ]:
print(f"The trapped ion axial phonon mode frequencys are:\n {res['phonon_freq']}\n")
print(f"The trapped ion axial Lamb-Dicke parameters are:\n {res['lamb_dicke']}\n")
print(f"infidelity is: {res['infidelity']}\n")
print(unitary)

同时用户也可以通过调用量脉 `Plot.plotIonPosition()` 画图模块可视化地呈现量子比特位置

In [ ]:
Plot.plotIonPosition(res['ion_pos'])

以及形成 Mølmer-Sørensen 门的脉冲波形

In [ ]:
Plot.plotPulse([res['time']], [res['omega']],
                title=['Square pulse for Molmer-Sorensen gate in trapped ion'],
                xLabel=r'Time ($\mu$s)', yLabel=['Rabi frequency (a.u)'], color=['blue'])


同时，除了实验上常用的方波脉冲，量脉也支持傅里叶正弦波脉冲。我们同样地，通过传输定义好的参数来求解傅里叶正弦波形成的 Mølmer-Sørensen 门时相关的脉冲：

In [ ]:
# Choose the pulse waveform, where we support squareWave and sinWave
pulseWave = "sinWave"

In [ ]:
res, unitary = runIonMS(ionNumber=ionNumber,
               atomMass=atomMass,
               tg=tgate,
               omega=(omegaXY, omegaZ),
               ionIndex=(ionM, ionN),
               pulseWave=pulseWave,
               phononMode=phononMode)

用户通过如下命令可以方便地打印量子比特位置、系统哈密顿量参数等信息

In [ ]:
print(f"The trapped ion axial phonon mode frequencys are:\n {res['phonon_freq']}\n")
print(f"The trapped ion axial Lamb-Dicke parameters are:\n {res['lamb_dicke']}\n")
print(f"infidelity is: {res['infidelity']}\n")
print(unitary)

并给出量子比特位置


In [ ]:
Plot.plotIonPosition(res['ion_pos'])

和形成 Mølmer-Sørensen 门的傅里叶正弦脉冲波形：

In [ ]:
Plot.plotPulse([res['time']], [res['omega']],
                title=['Sin pulse for Molmer-Sorensen gate in trapped ion'],
                xLabel=r'Time ($\mu$s)', yLabel=['Rabi frequency (a.u)'], color=['blue'])

通过量脉离子阱 `runIonMS()` 模块，我们得到了在傅里叶正弦基下形成 Mølmer-Sørensen 门的脉冲信息以及相应的系统哈密顿量关键参数

## 总结

通过使用量脉，我们可以方便地求解一维离子阱的哈密顿量参数信息，包括离子位置标定信息、声子振动模式和系统 Lamb-Dicke 系数，进一步的，通过量脉离子阱模块，我们可以方便地计算得到构成单量子比特门以及 Mølmer-Sørensen 门的脉冲波形。用户可以通过点击链接 [tutorial-iontrap.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/CN/tutorial-ion-trap-single-and-two-qubit-gate-cn.ipynb) 跳转到此 Jupyter Notebook 文档相应的 GitHub 页面并且运行这个程序。

我们鼓励用户尝试不同于本教程的参数值以进一步探索量脉离子阱模块的功能。

## 参考文献
\[1\] [Wright, K., et al. "Benchmarking an 11-qubit quantum computer." *Nature communications* 10.1 (2019): 1-6.](https://www.nature.com/articles/s41467-019-13534-2)

\[2\] [Bruzewicz, Colin D., et al. "Trapped-ion quantum computing: Progress and challenges." *Applied Physics Reviews* 6.2 (2019): 021314.](https://aip.scitation.org/doi/abs/10.1063/1.5088164)

\[3\] [Häffner, Hartmut, Christian F. Roos, and Rainer Blatt. "Quantum computing with trapped ions." *Physics reports* 469.4 (2008): 155-203.](https://www.sciencedirect.com/science/article/abs/pii/S0370157308003463)

\[4\] [Sørensen, Anders, and Klaus Mølmer. "Quantum computation with ions in thermal motion." *Physical review letters* 82.9 (1999): 1971.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.82.1971)
